In [ ]:
!nvidia-smi

Fri Aug 18 18:08:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    41W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 88.7 MB/s eta 0:00:0

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.0 MB/s eta 0:00:00
Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Successfully uninstalled transformers-4.31.0
Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Successfully uninstalled accelerate-0.21.0
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)


In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Ваш код делает следующее:

Импортирует AutoModelForSeq2SeqLM и AutoTokenizer из библиотеки transformers. Эти компоненты используются для работы с предварительно обученными моделями машинного перевода и токенизаторами из библиотеки Hugging Face.

Вы определяете переменную device, которая будет содержать значение "cuda", если у вас имеется поддержка CUDA (что указывает на наличие GPU), иначе она будет содержать значение "cpu". Это стандартный подход для определения устройства, на котором будет выполняться тренировка или инференс в PyTorch.

В последней строке вы просто выводите значение переменной device.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Ваш код продолжает работу с библиотекой transformers от Hugging Face. Вот что происходит по шагам:

Вы определяете строковую переменную model_ckpt, которая содержит путь к предварительно обученной модели PEGASUS для задачи суммаризации на датасете CNN/DailyMail.

Вы создаете токенизатор, используя AutoTokenizer.from_pretrained(model_ckpt). Этот токенизатор будет нужен для преобразования ваших текстовых данных в формат, понимаемый моделью.

Вы загружаете предварительно обученную модель AutoModelForSeq2SeqLM из указанного чекпоинта. После этого, вы перемещаете модель на выбранное устройство (GPU или CPU) с помощью метода .to(device).

Теперь у вас есть всё необходимое для выполнения задачи суммаризации текста с использованием модели PEGASUS!

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip - эта команда загружает zip-архив с данными для суммаризации из указанной ссылки.

!unzip summarizer-data.zip - после загрузки архива, эта команда извлекает содержимое архива в текущую директорию.

In [ ]:
#dowload & unzip data

!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

--2023-08-18 18:10:45--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2023-08-18 18:10:46--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.03s   

2023-08-18 18:10:46 (219 MB/s) - ‘summarizer-data.zip’ saved [7903594/7903

вы пытаетесь загрузить набор данных, сохраненный на диске, с использованием функции load_from_disk. Эта функция, вероятно, из библиотеки datasets от Hugging Face.

In [ ]:
dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

Этот код предназначен для вывода информации о загруженном наборе данных dataset_samsum.

Подробно:

split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]: Вы вычисляете длину каждого подмножества (например, 'train', 'test', 'validation') в вашем наборе данных и сохраняете их в списке split_lengths.

print(f"Split lengths: {split_lengths}"): Выводите длины подмножеств на экран.

print(f"Features: {dataset_samsum['train'].column_names}"): Показываете имена столбцов (или "features") для тренировочного подмножества.

print("\nDialogue:"): Просто печатаете заголовок "Dialogue".

print(dataset_samsum["test"][1]["dialogue"]): Выводите диалог из второго элемента тестового подмножества.

print("\nSummary:"): Печатаете заголовок "Summary".

print(dataset_samsum["test"][1]["summary"]): Выводите резюме из второго элемента тестового подмножества.

В итоге этот код дает представление о структуре вашего набора данных, показывая длину каждого подмножества, имена столбцов и пример диалога со сводкой из тестового подмножества.

Если у вас возникнут вопросы или если вы хотите узнать больше о содержимом вашего набора данных, дайте знать!

In [ ]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


Давайте разберемся с функцией convert_examples_to_features.

Эта функция принимает на вход пакет примеров (example_batch), который, вероятно, состоит из диалогов и их сводок. Задача функции — преобразовать эти примеры в формат, подходящий для обучения или предсказания модели PEGASUS.

input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True ):

Здесь используется токенизатор, чтобы преобразовать диалоги из пакета примеров в токенизированный формат.
С параметром max_length = 1024 длина каждого диалога ограничивается 1024 токенами.
Если диалог превышает этот лимит, truncation = True обрезает его.
with tokenizer.as_target_tokenizer()::

Это контекстное управление гарантирует, что токенизатор используется в режиме "целевого" токенизатора. Это может включать специфические правила или токены, используемые для целевых последовательностей (в данном контексте — сводки).
target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True ):

Токенизация сводок с аналогичными настройками, как и для диалогов, но с максимальной длиной 128 токенов.
Возвращаемый словарь содержит:

input_ids: ID токенов для входных диалогов.
attention_mask: Маска внимания, которая указывает, какие токены входного диалога являются актуальными, а какие — заполнителями.
labels: ID токенов для сводок. Эти метки будут использоваться в качестве "правильных ответов" при обучении модели.
Эта функция удобна для преобразования пакетов данных перед их подачей в модель во время обучения или оценки.

Токенизация — это процесс преобразования текста в последовательность "токенов". Токенами могут быть слова, символы, подслова или даже буквы. Токенизация является одним из первых этапов в большинстве задач обработки текста.

Пример токенизации:
Для предложения "Я люблю программировать" токенизация на уровне слов вернет: ["Я", "люблю", "программировать"].

Однако токенизация может быть более сложной, особенно в нейронных моделях обработки естественного языка. В этих моделях слова могут быть разбиты на подслова или символы, особенно если слово редкое или вовсе не встречалось в обучающем датасете.

Токенизаторы, как tokenizer в вашем коде, обычно делают следующее:

Разбивают текст на токены: Как уже было упомянуто, это может быть на уровне слов, подслова или символов.
Преобразуют токены в идентификаторы: Каждому уникальному токену сопоставляется уникальное число (ID). Эти идентификаторы затем используются для обучения или предсказания в моделях машинного обучения.
Добавляют специальные токены: Например, токены начала и конца последовательности, которые могут быть необходимы для некоторых архитектур моделей, таких как трансформеры.
Создают маски внимания: Это бинарные маски, которые указывают модели, на какие токены следует обращать внимание, а какие игнорировать (например, заполнители после конца реального текста).
В вашем коде tokenizer скорее всего представляет собой предварительно обученный токенизатор из библиотеки Hugging Face. Он знает, как правильно токенизировать текст для конкретной модели, с которой вы работаете (в вашем случае, PEGASUS).

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Содержит тренировочные примеры этого набора данных. Столбцы и кол-во строк

In [ ]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

DataCollatorForSeq2Seq используется для подготовки данных для обучения seq2seq моделей.
Создаем экземпляр класса DataCollatorForSeq2Seq, передавая ему tokenizer и model_pegasus.


In [ ]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

Из библиотеки transformers импортируются классы TrainingArguments и Trainer. TrainingArguments предназначен для задания параметров обучения, в то время как Trainer используется для управления самим процессом обучения. Далее создается экземпляр класса TrainingArguments с параметрами: output_dir указывает каталог для сохранения модели и логов; num_train_epochs задает количество эпох обучения; warmup_steps определяет количество шагов разогрева, что полезно для стабильности обучения с некоторыми оптимизаторами; per_device_train_batch_size и per_device_eval_batch_size задают размеры пакетов данных для обучения и оценки соответственно для каждого устройства; weight_decay используется для регуляризации путем уменьшения веса; logging_steps определяет, как часто будут записываться логи; evaluation_strategy задает стратегию оценки, в данном случае оценка будет проводиться каждые eval_steps шагов; save_steps определяет, как часто модель будет сохраняться, при этом значение установлено довольно высоким для редкого сохранения; и наконец, gradient_accumulation_steps указывает, сколько шагов будет накапливаться градиент перед обновлением весов, что может быть полезным при использовании маленьких размеров пакетов данных.

In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

Код создаёт объект Trainer, который представляет собой инструмент из библиотеки transformers для обучения моделей. Этот объект обеспечивает большую часть стандартного функционала обучения, такого как итерация по данным, оптимизация модели и её оценка.

Вот подробное разъяснение каждого аргумента:

model=model_pegasus: Здесь передается модель, которую вы хотите обучить. В вашем случае это PEGASUS, предположительно для задачи суммирования.

args=trainer_args: Это аргументы обучения, которые были заданы ранее с помощью TrainingArguments.

tokenizer=tokenizer: Токенизатор, используемый для преобразования текстовых данных в формат, подходящий для модели.

data_collator=seq2seq_data_collator: Данный аргумент определяет, как будут формироваться пакеты из данных. В вашем случае используется специальный сборщик данных для seq2seq задач.

train_dataset=dataset_samsum_pt["test"]: Набор данных для обучения. Странно, что вы используете тестовый набор данных для обучения, обычно это делается с обучающим набором.

eval_dataset=dataset_samsum_pt["validation"]: Набор данных для оценки модели в процессе обучения.

In [ ]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])

Данный код запускает процесс обучения модели с использованием объекта Trainer, который был создан ранее.

trainer.train() выполняет следующие действия:

Инициализирует оптимизатор и планировщик шага обучения (если они не были предоставлены пользователем).
Проходит по указанному вами набору данных для обучения (train_dataset), используя параметры обучения, указанные в TrainingArguments.
Во время обучения, модель будет обновлять свои веса на каждом шаге, используя выбранный вами алгоритм оптимизации.
Если указаны параметры для логирования и оценки, Trainer будет также регулярно записывать результаты и оценивать производительность модели на валидационном наборе данных.
По завершении обучения, модель и её параметры сохраняются в указанной директории (output_dir в TrainingArguments).

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.000213454751407, metrics={'train_runtime': 97.4916, 'train_samples_per_second': 8.401, 'train_steps_per_second': 0.523, 'total_flos': 313317832187904.0, 'train_loss': 3.000213454751407, 'epoch': 1.0})

Функция generate_batch_sized_chunks принимает список элементов и размер батча в качестве входных данных. Она разбивает список на части (батчи) заданного размера и возвращает эти части последовательно. Другая функция, calculate_metric_on_test_ds, принимает датасет, метрику, модель, токенизатор и ряд других параметров. Сначала эта функция разбивает тексты и их соответствующие сводки из датасета на батчи. Затем она итерируется по этим батчам, токенизируя тексты и генерируя сводки с помощью переданной модели. Параметр length_penalty в функции генерации убеждается, что модель не создает слишком длинные последовательности. После генерации сводок функция декодирует их, превращая их обратно в читаемый текст. Эти декодированные сводки затем добавляются в метрику вместе с их истинными значениями для оценки. После прохождения по всем батчам функция вычисляет и возвращает значение метрики (например, ROUGE) для оценки качества сгенерированных сводок по сравнению с истинными сводками.

In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

Этот код связан с оценкой качества генерации текста, часто используемой в задачах автоматического суммирования.

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]: Здесь вы определяете список метрик ROUGE, которые хотите рассмотреть.

rouge1: оценка на основе унограмм.
rouge2: оценка на основе биграмм.
rougeL: наибольшая общая подпоследовательность.
rougeLsum: основана на наибольших общих подпоследовательностях предложений.
rouge_metric = load_metric('rouge'): Здесь вы загружаете метрику ROUGE из библиотеки datasets от Hugging Face. Этот инструмент позволяет вам автоматически вычислять значения ROUGE для сгенерированных текстов по сравнению с оригинальными (например, для сравнения сгенерированных сводок с реальными сводками).

Далее, вы можете использовать rouge_metric для вычисления ROUGE-оценок между референтными и сгенерированными текстами, как это было показано в предыдущем коде, который вы предоставили.

In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

<ipython-input-18-5a43aadd1b0e>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


Вызов функции calculate_metric_on_test_ds:

dataset_samsum['test'][0:10]: здесь вы выбираете первые 10 элементов из тестового набора данных samsum.
rouge_metric: передаете метрику ROUGE, которую вы загрузили ранее.
trainer.model: передаете вашу модель, которая была инициализирована и, возможно, обучена с использованием объекта Trainer.
tokenizer: передаете токенизатор, который используется для преобразования ваших текстовых данных в формат, пригодный для модели.
batch_size = 2: указываете, что хотите обрабатывать данные батчами по 2 элемента.
column_text = 'dialogue': указываете, что текстовые данные находятся в столбце dialogue вашего датасета.
column_summary = 'summary': указываете, что референтные сводки находятся в столбце summary.
Получение словаря ROUGE-оценок:

dict((rn, score[rn].mid.fmeasure) for rn in rouge_names): это создает словарь, где ключами являются имена метрик ROUGE (например, "rouge1"), а значениями - средние значения F-меры для каждой из метрик на основе результата функции calculate_metric_on_test_ds.
Преобразование результатов в DataFrame:

pd.DataFrame(rouge_dict, index = [f'pegasus']): это преобразует словарь ROUGE-оценок в DataFrame от pandas с индексом pegasus.
Этот код позволяет вам быстро и удобно оценить качество вашей модели pegasus на небольшой выборке тестового набора данных с использованием метрики ROUGE. В результате вы получите таблицу DataFrame, где строки представляют модель, а столбцы - значения F-меры для различных метрик ROUGE.

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|██████████| 5/5 [00:11<00:00,  2.30s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.021153,0.0,0.021227,0.020807


метод save_pretrained из библиотеки Hugging Face Transformers для сохранения обученной модели на вашем локальном диске.

Команда model_pegasus.save_pretrained("pegasus-samsum-model") делает следующее:

Сохраняет конфигурацию модели и веса модели в указанную директорию ("pegasus-samsum-model" в вашем случае).
После сохранения, вы можете загрузить модель позже из этой директории с помощью from_pretrained метода.
Простыми словами, вы сохранили свою обученную модель "pegasus" для задачи "samsum", чтобы позже использовать её или передать другим.

In [ ]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

Вы использовали метод save_pretrained для сохранения токенизатора, который ассоциирован с вашей моделью. Вот что делает данная команда:

tokenizer.save_pretrained("tokenizer"): Эта команда сохраняет файлы конфигурации и словарь токенизатора в указанную директорию ("tokenizer" в вашем случае).
После выполнения этой команды в директории "tokenizer" появятся следующие файлы (в зависимости от используемого типа токенизатора):

tokenizer_config.json: содержит конфигурацию токенизатора.
vocab.json или vocab.txt: содержит словарь токенизатора.
merges.txt (если используется Byte-Pair Encoding или BPE токенизация): содержит правила слияния для BPE.
special_tokens_map.json: содержит информацию о специальных токенах.
Теперь у вас есть возможность переносить токенизатор или загружать его позже из этой директории с помощью метода from_pretrained. Это особенно полезно, если вы планируете распределять свою модель или токенизатор или использовать их в другом месте.

In [ ]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

Вы предоставили кортеж файлов, который обычно сохраняется при использовании метода save_pretrained для токенизатора из библиотеки Hugging Face Transformers. Вот что означает каждый из этих файлов:

tokenizer_config.json: Этот файл содержит различные параметры конфигурации токенизатора, такие как тип токенизатора (например, BPE, WordPiece и др.), а также другие специфические параметры.

special_tokens_map.json: Этот файл содержит информацию о специальных токенах, таких как токены начала и конца последовательности, токены маскирования и другие. Эти токены обычно используются моделями для специфических задач (например, обучение с применением маски в BERT).

spiece.model: Этот файл, характерный для токенизаторов типа SentencePiece, содержит обученную модель токенизации. SentencePiece - это токенизатор, который может использовать разные методы токенизации, такие как BPE или Unigram.

added_tokens.json: Если вы добавляли какие-либо пользовательские токены к токенизатору после его инициализации, этот файл будет содержать информацию о них. Это полезно, когда вам нужно сохранить токенизатор со специальными токенами, добавленными для конкретного проекта.

tokenizer.json: Это общий файл токенизатора, который может использоваться для загрузки всей информации о токенизации, включая словарь, правила и конфигурации.

Таким образом, благодаря этим файлам, вы можете переносить свой токенизатор между разными проектами или машинами без потери какой-либо информации или конфигурации.

In [ ]:
('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

Вы используете метод from_pretrained класса AutoTokenizer из библиотеки Hugging Face Transformers для загрузки ранее сохраненного токенизатора из директории.

Подробнее:

AutoTokenizer: Этот класс позволяет автоматически определить и загрузить нужный тип токенизатора на основе предоставленной директории или имени предварительно обученной модели. Таким образом, он обеспечивает удобство при работе с различными моделями, не требуя явного указания типа токенизатора.

from_pretrained("/content/tokenizer"): Этот метод загружает токенизатор из указанной директории. В вашем случае, вы указали директорию "/content/tokenizer", в которой ранее был сохранен токенизатор.

После выполнения этой команды, объект tokenizer будет содержать загруженный токенизатор, который теперь можно использовать для токенизации текста, преобразования текста в токены, декодирования и т.д.

In [ ]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

Вы используете функцию pipeline из библиотеки Hugging Face Transformers для выполнения задачи автоматического реферирования. Вот разбор предоставленного кода:

gen_kwargs: Это словарь с аргументами для генерации текста. Вы устанавливаете length_penalty для регулирования длины сгенерированного текста, num_beams для установки количества лучей в beam search и max_length для установки максимальной длины сгенерированного текста.

sample_text и reference: Вы извлекаете пример диалога и соответствующее ему реферирование из тестового набора данных dataset_samsum.

pipe: Вы создаете конвейер (pipeline) для задачи реферирования. Вы используете модель, сохраненную в директории "pegasus-samsum-model", и ранее загруженный токенизатор.

print("Dialogue:"): Печатаете исходный диалог.

print("\nReference Summary:"): Печатаете оригинальное реферирование для данного диалога (т.е. то, которое было предоставлено в наборе данных).

print("\nModel Summary:"): Печатаете реферирование, сгенерированное вашей моделью. Используя созданный ранее конвейер, вы передаете в него исходный диалог и аргументы генерации, чтобы получить реферирование.

После выполнения этого кода на экран будет выведен исходный диалог, оригинальное реферирование из набора данных и реферирование, созданное вашей моделью. Это позволит оценить, насколько качественно модель выполняет задачу реферирования для данного примера.







In [ ]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
